## Finding the most depressing song with Spotify

inspired by http://rcharlie.com/2017-02-16-fitteR-happieR/#analysis

### Getting track list for an artist from Spotify API
https://developer.spotify.com/web-api/


In [228]:
import requests
req= requests.get(url='https://api.spotify.com/v1/search', params= {'q':'radiohead', 'type':'artist'}
                 )
resp=req.json()

In [229]:
import pprint as pp
pp.pprint(resp)

{'artists': {'href': 'https://api.spotify.com/v1/search?query=radiohead&type=artist&offset=0&limit=20',
             'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
                        'followers': {'href': None, 'total': 1990587},
                        'genres': ['alternative rock',
                                   'indie rock',
                                   'melancholia',
                                   'permanent wave',
                                   'rock'],
                        'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
                        'id': '4Z8W4fKeB5YxbusRsdQVPb',
                        'images': [{'height': 640,
                                    'url': 'https://i.scdn.co/image/afcd616e1ef2d2786f47b3b4a8a6aeea24a72adc',
                                    'width': 640},
                                   {'height': 320,
                                    'url': 'https

In [189]:
## parse json response and combine needed artist info (name and URI) into dataframe
import pandas as pd
from pandas.io.json import json_normalize
artists_df=json_normalize(resp['artists']['items'])
artists_df=artists_df[['name', 'uri']]
artists_df['uri']= artists_df['uri'].apply(lambda x : x.replace('spotify:artist:', ''))
artists_df.head()

,name,uri
0,Radiohead,4Z8W4fKeB5YxbusRsdQVPb
1,Radiohead Tribute Band,0ADkBHZhR2cVfANgK5gHQO


In [190]:
'https://api.spotify.com/v1/artists/{artist_uri}/albums'.format(artist_uri=artists_df.loc[0, 'uri'])

'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb/albums'

In [191]:
##using artists URI get artists albums and songs 
req= requests.get(url='https://api.spotify.com/v1/artists/{artist_uri}/albums'.format(artist_uri=artists_df.loc[0, 'uri']),
                 params= {'limit':50})
resp_albums=req.json()
pp.pprint(resp_albums)

##check the TOTAL!

{'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb/albums?offset=0&limit=50&album_type=single,album,compilation,appears_on,ep',
 'items': [{'album_type': 'album',
            'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
                         'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
                         'id': '4Z8W4fKeB5YxbusRsdQVPb',
                         'name': 'Radiohead',
                         'type': 'artist',
                         'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
            'available_markets': ['AD',
                                  'AR',
                                  'AT',
                                  'AU',
                                  'BE',
                                  'BG',
                                  'BO',
                                  'BR',
                                  'CA',
                                

In [192]:
albums_df=json_normalize(resp_albums['items'])
albums_df=albums_df[['id','type', 'name' ]]
albums_df.head()

,id,type,name
0,6vuykQgDLUCiZ7YggIpLM9,album,A Moon Shaped Pool
1,47xaqCsJcYFWqD1gwujl1T,album,TKOL RMX 1234567
2,1DBkJIEoeHrTX4WCBQGcCi,album,The King Of Limbs
3,7eyQXxuf2nGj9d2367Gi5f,album,In Rainbows
4,36lJLPoPPOKNFddTAcirnc,album,In Rainbows Disk 2


In [230]:
## we need to support pagination
def get_artist_albums(artist_id):
    columns=['id','type', 'name' ]
    albums_df=pd.DataFrame()
    limit=50 #set the number of albums to retrieve from API, 50 is the max
    offset=0 #The index of the first album to return. Default: 0 (i.e., the first album)
    total=limit
    while total> offset:
        req= requests.get(url='https://api.spotify.com/v1/artists/{artist_uri}/albums'.format(artist_uri=artist_id),
                 params= {'limit':limit, 'offset':offset})
        resp_albums=req.json()
        #pp.pprint(json_normalize(resp_albums['items'])[columns],)
        albums_df=albums_df.append(json_normalize(resp_albums['items'])[columns], ignore_index=True)
        #pp.pprint(albums_df)
        ##read the total from response and update offset
        total=resp_albums['total']
        offset=offset + limit 
        print ('TOTAL: %d, OFFSET: %d' % (total, offset))
    return albums_df 

all_albums=get_artist_albums(artists_df.loc[0, 'uri'])
print (all_albums.shape)
all_albums.head(100)


TOTAL: 94, OFFSET: 50
TOTAL: 94, OFFSET: 100
(94, 3)


,id,type,name
0,6vuykQgDLUCiZ7YggIpLM9,album,A Moon Shaped Pool
1,47xaqCsJcYFWqD1gwujl1T,album,TKOL RMX 1234567
2,1DBkJIEoeHrTX4WCBQGcCi,album,The King Of Limbs
3,7eyQXxuf2nGj9d2367Gi5f,album,In Rainbows
4,36lJLPoPPOKNFddTAcirnc,album,In Rainbows Disk 2
5,6Eo5EkmdLvZrONzi046iC2,album,Com Lag: 2+2=5
6,1oW3v5Har9mvXnGk0x4fHm,album,Hail To the Thief
7,6svTt5o2lUgIrgYDKVmdnD,album,I Might Be Wrong
8,6V9YnBmFjWmXCBaUVRCVXP,album,Amnesiac
9,19RUXBFyM4PpmrLRdtqWbp,album,Kid A


In [203]:
##Having all the album ids we can pull the full track list for each album
def get_tracks(album_id):
    req_tracks=requests.get(url='https://api.spotify.com/v1/albums/{id}/tracks'.format(id=album_id), 
                           params={'limit':50}) ##here make an assumption that track list is <=50 songs
    json_resp=req_tracks.json()
    
    #list(map(requests.get(url='https://api.spotify.com/v1/albums/{id}/tracks'.format(id=album_id), 
    #                       params={'limit':50})))
    return json_resp['items']


all_tracks= list(map(get_tracks,all_albums.id))[0]



In [204]:
pp.pprint(all_tracks[0])

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
              'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
              'id': '4Z8W4fKeB5YxbusRsdQVPb',
              'name': 'Radiohead',
              'type': 'artist',
              'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
 'available_markets': ['AD',
                       'AR',
                       'AT',
                       'AU',
                       'BE',
                       'BG',
                       'BO',
                       'BR',
                       'CA',
                       'CH',
                       'CL',
                       'CO',
                       'CR',
                       'CY',
                       'CZ',
                       'DE',
                       'DK',
                       'DO',
                       'EC',
                       'EE',
                       'ES',
                       'FI',


In [282]:
tracks_df=json_normalize(all_tracks)
tracks_df.id.head()
artists=[y['name'] for y in [x['artists'][0] for x in all_tracks]]
#json_normalize(all_tracks[0]['artists'])

##FILTER OUT compilation and singles and songs by other artists
#for x, y in tracks_df.artists.iteritems():
#    print((y))
all_art=[y['name'] for y in [for x in tracks_df.artists]]
all_art
#[ in tracks_df.artists]
for x in tracks_df.artists:
    for y in x:
        print()
#df=pd.DataFrame(json_normalize(artists))
##filter out artist
#df.head()

SyntaxError: invalid syntax (<ipython-input-282-62353ff34d04>, line 9)

### Get Audio Features for a Track
Authorization Required. A valid access token from the Spotify Accounts service: see the Web API Authorization Guide for details:
https://developer.spotify.com/web-api/authorization-guide/

Create spotify Developers account (you need to be registered Spotify user) https://developer.spotify.com/my-applications/#!/applications 

In [210]:
##request authentication token using your Client Id and Client Secret
import config
data = {'key':'value'}
#Authorization Required. Base 64 encoded string that contains the client ID and client secret key. 
#The field must have the format: Authorization: Basic <base64 encoded client_id:client_secret>
from requests.auth import HTTPBasicAuth

auth= requests.post('https://accounts.spotify.com/api/token',data={'grant_type':'client_credentials'},
                         auth=HTTPBasicAuth(config.SpotifyClient.client_id, config.SpotifyClient.client_secret))

##read the status code - 200 means success
auth.status_code

200

In [221]:
##using access token you can now autorize and get audio features for a track 
auth_token=auth.json()['access_token']
track_audio_features= list(map(lambda x: requests.get('https://api.spotify.com/v1/audio-features/{track_id}'.format(track_id=x), 
                                        headers={"Authorization": "Bearer %s" %auth_token}).json(), tracks_df.id ))
 


In [222]:
pp.pprint(track_audio_features)

[{'acousticness': 0.303,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3pcCifdPTc2BbqmWpEhtUd',
  'danceability': 0.541,
  'duration_ms': 220609,
  'energy': 0.847,
  'id': '3pcCifdPTc2BbqmWpEhtUd',
  'instrumentalness': 0.271,
  'key': 11,
  'liveness': 0.109,
  'loudness': -6.52,
  'mode': 1,
  'speechiness': 0.0297,
  'tempo': 148.937,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/3pcCifdPTc2BbqmWpEhtUd',
  'type': 'audio_features',
  'uri': 'spotify:track:3pcCifdPTc2BbqmWpEhtUd',
  'valence': 0.625},
 {'acousticness': 0.968,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1uRxyAup7OYrlh2SHJb80N',
  'danceability': 0.299,
  'duration_ms': 384439,
  'energy': 0.263,
  'id': '1uRxyAup7OYrlh2SHJb80N',
  'instrumentalness': 0.857,
  'key': 9,
  'liveness': 0.126,
  'loudness': -13.207,
  'mode': 0,
  'speechiness': 0.0323,
  'tempo': 137.848,
  'time_signature': 3,
  'track_href': 'https://api.spotify.com/v1/tracks/1uRxyAup7OYrlh2S

In [226]:
audio_feat_df=json_normalize(track_audio_features)
audio_feat_df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.303,https://api.spotify.com/v1/audio-analysis/3pcC...,0.541,220609,0.847,3pcCifdPTc2BbqmWpEhtUd,0.271,11,0.109,-6.520,1,0.0297,148.937,4,https://api.spotify.com/v1/tracks/3pcCifdPTc2B...,audio_features,spotify:track:3pcCifdPTc2BbqmWpEhtUd,0.625
1,0.968,https://api.spotify.com/v1/audio-analysis/1uRx...,0.299,384439,0.263,1uRxyAup7OYrlh2SHJb80N,0.857,9,0.126,-13.207,0,0.0323,137.848,3,https://api.spotify.com/v1/tracks/1uRxyAup7OYr...,audio_features,spotify:track:1uRxyAup7OYrlh2SHJb80N,0.115
2,0.668,https://api.spotify.com/v1/audio-analysis/5rIh...,0.560,281011,0.500,5rIhBK9aaVMck0W2YtOwci,0.833,2,0.117,-10.829,0,0.0272,139.174,4,https://api.spotify.com/v1/tracks/5rIhBK9aaVMc...,audio_features,spotify:track:5rIhBK9aaVMck0W2YtOwci,0.258
3,0.868,https://api.spotify.com/v1/audio-analysis/1kBG...,0.561,224698,0.282,1kBGeOp1CDUHVdbK4ergqo,0.529,0,0.110,-13.241,1,0.0286,112.011,1,https://api.spotify.com/v1/tracks/1kBGeOp1CDUH...,audio_features,spotify:track:1kBGeOp1CDUHVdbK4ergqo,0.153
4,0.353,https://api.spotify.com/v1/audio-analysis/4CzT...,0.426,367349,0.661,4CzTgOmc3Sdm4EgKQWzjQl,0.940,0,0.619,-9.877,1,0.0503,151.191,3,https://api.spotify.com/v1/tracks/4CzTgOmc3Sdm...,audio_features,spotify:track:4CzTgOmc3Sdm4EgKQWzjQl,0.179


In [227]:
af_columns=['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'album_release_year',
                    'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'track_number']